
<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [2]:
!pip install bitstring
!pip install numpy
import bitstring as bs
import numpy as np

You should consider upgrading via the 'c:\users\nicer\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\nicer\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant,bits_exp):
    # Algoritmo de representación de punto flotante modificada.
    n = bs.pack('>d', x) # Pasar el numero original a punto flotante
    b = n.bin

    # Signo
    signo = b[0]

    # Trabajar la mantisa
    mantisa = n[12:12+bits_mant]
    mantisa_b = n[12:12+bits_mant].uint
    if bits_mant < 52: # Si no se necesita la mantisa original completa
        if n[12 + bits_mant - 1] == '1' or int('0b' + b[bits_mant+12:], 2) > 2**(51-bits_mant):
            if mantisa_b == 2**bits_mant - 1:
                mantisa_b = 0
                n[12:12+bits_mant] = mantisa_b
                exp = n[1:12].uint
                exp += 1
                n[1:12] = exp
            else:
                mantisa_b += 1
                n[12:12+bits_mant] = mantisa_b
    mantisa = n[12:12+bits_mant].bin

    # Trabajar el exponente nuevo 
    exp = int('0b' + b[1:12], 2) - 1023 + ((2**(bits_exp-1))-1) # Calcular el valor decimal del nuevo exponente 
    exp = bin(exp)[2:] # Pasarlo a binario nuevamente
    exp = (bits_exp-len(exp))*'0' + exp # Agregarle 0's si hace falta

    # Rellenando el resto de mantisa con 0
    # para pasar nuevamente a decimal
    n[12+bits_mant:] = 0
    
    result = 0

    # Revisar casos especiales
    equal = all_equal(exp) 
    if equal[0]:
        if equal[1] == '1':
            if signo == '0':
                result = np.inf
            else:
                if all_equal(n[12:12+bits_mant])[0]:
                    result = -np.inf
                else:
                    result = np.nan
    else:
        result = n.float

    return  result

"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    return x_roots


### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\log(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


In [2]:
!pip install numpy
import numpy as np

You should consider upgrading via the 'c:\users\nicer\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [46]:
def fpi(g, x0, k, tol):
    x = np.empty(k+1)
    x[0] = x0
    error_i = np.nan
    
    print(' i |     x(i)     |    x(i+1)    ||x(i+1)-x(i)| | e_i/e_{i-1}')
    print('--------------------------------------------------------------')
    
    for i in range(k):
        x[i+1] = g(x[i])
        error_iminus1 = error_i
        error_i = abs(x[i+1]-x[i])
        print('%2d | %.10f | %.10f | %.10f | %.10f' %
        (i,x[i],x[i+1],error_i,error_i/error_iminus1))
        
    return x[-1]

def newton_method(f, fp, x0, rel_error=1e-8, m=1):
    #Initialization of hybrid error and absolute
    hybrid_error = 100
    error_i = np.inf
    
    print('i |     x(i)     |     x(i+1)   |      |x(i+1)-x(i)|     |  e_i/e_{i-1} | e_i/e_{i-1}^2')
    print('----------------------------------------------------------------------------------------')
    
    #Iteration counter
    i = 1
    while (hybrid_error > rel_error and hybrid_error < 1e12 and i < 1e4):
        #Newton's iteration
        x1 = x0-m*f(x0)/fp(x0)
        
        #Checking if root was found
        if f(x1) == 0.0:
            hybrid_error = 0.0
            break
        
        #Computation of hybrid error
        hybrid_error = abs(x1-x0)/np.max([abs(x1),1e-12])
        
        #Computation of absolute error
        error_iminus1 = error_i
        error_i = abs(x1-x0)
        
        #Increasing counter
        i += 1
        
        #Showing some info
        print("%d | %.10f | %.10f | %.20f | %.10f | %.10f" % 
              (i, x0, x1, error_i, error_i/error_iminus1, error_i/(error_iminus1**2)))
        #Updating solution
        x0 = x1
    
    #Checking if solution was obtained
    if hybrid_error < rel_error: 
        return x1
    elif i>=1e4:
        print('Newton''s Method diverged. Too many iterations!!')
        return None
    else:
        print('Newton''s Method diverged!')
        return None

In [65]:
"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""

def raiz_de_oreman(alpha, beta, tol = 1e-10):
    # Su implementación.
    oreman = lambda x: (alpha * np.cos(x) + beta * np.sin(np.log(x**2)/2))
    oreman_derivada = lambda x: (-alpha*np.sin(x) + (beta/x)*np.cos(np.log(x**2)/2))

    g = lambda x: x - (oreman(x)/oreman_derivada(x))

    f = lambda x: np.exp(np.arcsin((-alpha/beta)*np.cos(x)))

    root = fpi(f, 25, 100, tol)

    return root

In [66]:
raiz_de_oreman(2, 4)

 i |     x(i)     |    x(i+1)    ||x(i+1)-x(i)| | e_i/e_{i-1}
--------------------------------------------------------------
 0 | 25.0000000000 | 0.5953968505 | 24.4046031495 | nan
 1 | 0.5953968505 | 0.6525918964 | 0.0571950459 | 0.0023436171
 2 | 0.6525918964 | 0.6646297615 | 0.0120378650 | 0.2104704146
 3 | 0.6646297615 | 0.6673006302 | 0.0026708687 | 0.2218722946
 4 | 0.6673006302 | 0.6678996788 | 0.0005990486 | 0.2242898047
 5 | 0.6678996788 | 0.6680343612 | 0.0001346823 | 0.2248270146
 6 | 0.6680343612 | 0.6680646576 | 0.0000302965 | 0.2249475467
 7 | 0.6680646576 | 0.6680714736 | 0.0000068159 | 0.2249746477
 8 | 0.6680714736 | 0.6680730070 | 0.0000015335 | 0.2249807441
 9 | 0.6680730070 | 0.6680733520 | 0.0000003450 | 0.2249821156
10 | 0.6680733520 | 0.6680734296 | 0.0000000776 | 0.2249824240
11 | 0.6680734296 | 0.6680734471 | 0.0000000175 | 0.2249824933
12 | 0.6680734471 | 0.6680734510 | 0.0000000039 | 0.2249825124
13 | 0.6680734510 | 0.6680734519 | 0.0000000009 | 0.2249825329


0.6680734521591395

# Referencias

Si corresponde